In [1]:
import os
import sys
sys.path.append(os.path.join(os.getcwd().replace("notebooks/pruning_quantization", "")))

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import MLP_Models, LSTM_Models, GRU_Models, CNN_models
from utils.benchmark import Benchmark
from compact.pruning import prune_mlp_model, prune_rnn_model, prune_cnn_model
from utils.plot import plot_fpr_tpr_roc_auc

In [ ]:
conf = Config()
load_model = GRU_Models()
model_conf = load_model.gru_pq
model = load_model.get_model(model_conf)
model.load()
print(conf.device)

Checkpoint loaded from /home/ubuntu/Network-Packet-ML-Model/checkpoint/large_model/gru_4.pth!
cpu


In [3]:
# data loader
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, model_type=load_model.type)

# create train, val and test dataloaders
train_dataset = NetworkDataset(X_train, y_train)
train_loader = DataLoader(train_dataset, conf.batch_size, shuffle=True)

val_dataset = NetworkDataset(X_val, y_val)
val_loader = DataLoader(val_dataset, conf.batch_size, shuffle=True)

dataset = NetworkDataset(X_test, y_test)
loader = DataLoader(dataset, conf.batch_size, shuffle=True)

In [4]:
# prune model
checkpoint_path = os.path.join(conf.checkpoint, "pruned_quantized", "pruned_" + model_conf["name"] + ".pth")
model.checkpoint_path = checkpoint_path

# new hidden sizes
if load_model.type == "mlp":
    pruned_model = prune_mlp_model(model.model)
elif load_model.type == "rnn":
    pruned_model = prune_rnn_model(model.model)
elif load_model.type == "cnn":
    pruned_model = prune_cnn_model(model.model)
else:
    raise ValueError("model type must be 'mlp', 'rnn' or 'cnn'!")
    

model.model = pruned_model.to(conf.device)
model.optimizer = torch.optim.AdamW(
    model.model.parameters(), 
    lr=conf.learning_rate, 
    weight_decay=conf.weight_decay
)
model.scheduler = torch.optim.lr_scheduler.ExponentialLR(
    model.optimizer, 
    gamma=conf.gamma
)

AttributeError: 'BatchNorm1d' object has no attribute 'elementwise_affine'

In [ ]:
# fine-tune pruned model
#metrics, train_loss, val_loss = model.train(train_loader, val_loader, epochs=1)
model.load()
# benchmark model
name = model_conf["name"]
result_path = os.path.join(conf.benchmark_dpu, "pruned_quantized_model", name + ".txt")
benchmark = Benchmark(model, loader, conf.batch_size, name, result_path)
benchmark()

# print and save result
benchmark.print_result()
benchmark.save()


Checkpoint loaded from /home/ubuntu/Network-Packet-ML-Model/checkpoint/pruned_quantized/pruned_lstm_pq.pth!


[W115 22:33:29.118142234 CPUAllocator.cpp:245] Memory block of unknown size was allocated before the profiling started, profiler results will not include the deallocation event


Benchmark - lstm_pq model:

Memory usage (MB):
Avg memory usage: 1.075MB
Peak memory usage: 15.822MB

Model inference latency on one batch (batch size = 1):
Avg latency: 111.364ms
Min latency: 95.006ms
Max latency: 203.614ms

Model inference throughput (batch size = 1):
Throughput: 8.63 samples/sec

Model inference CPU usage (number of logical cores) during runtime:
CPU runtime: 6.35 seconds
Average CPU usage: 5.90/8 cores

Model (lstm_pq) Macro-F1, Micro-F1 and Macro ROC AUC scores:
Macro-F1 score: 0.39
Micro-F1 score: 0.89
Macro ROC AUC score: nan





/home/ubuntu/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/ubuntu/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/ubuntu/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/ubuntu/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/home/ubuntu/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetri

In [ ]:
#roc_auc_path = os.path.join(os.getcwd().replace("notebooks/pruning_quantization", ""), "plots", "pruning_quantization", "roc_auc_" + "pruned_" + model_conf["name"] + ".png")
#plot_fpr_tpr_roc_auc(metrics[0], roc_auc_path)

In [ ]:
#model.save()